# Cartographie des points de charge IRVE recencées sur data.gouv.fr

## TODO

- [X] afficher un fond de carte
- [ ] faire une jolie carte
- [ ] version interactive

In [ ]:
import pandas as pd
import numpy as np

import holoviews as hv
from holoviews import opts
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, dynspread

import geoviews as gv
import geoviews.feature as gf

import datashader as ds
import datashader.transfer_functions as tf
from datashader.transfer_functions import shade, stack

hv.extension('bokeh', width=95)

In [ ]:
df = pd.read_csv("/tmp/out/bornes-IRVE.csv", sep=";")

In [ ]:
df = df[ pd.to_numeric(df['Xlongitude'], errors='coerce').notnull() ]
df = df[ pd.to_numeric(df['Ylatitude'], errors='coerce').notnull() ]
df = df[ pd.to_numeric(df['nbre_pdc'], errors='coerce').notnull() ]

In [ ]:
df['Xlongitude'] = pd.to_numeric(df['Xlongitude'])
df['Ylatitude'] = pd.to_numeric(df['Ylatitude'])
df['nbre_pdc'] = pd.to_numeric(df['nbre_pdc'])

In [ ]:
list(df)

In [ ]:
df.info()

In [ ]:
df[["Xlongitude","Ylatitude","nbre_pdc"]].head()

In [ ]:
(df['x'], df['y']) = ds.utils.lnglat_to_meters(df['Xlongitude'], df['Ylatitude'])

In [ ]:
bounds = [[-6.152344,42.504503],[8.701172,51.713416]]

cvs = ds.Canvas(plot_width=300, plot_height=300)
agg = cvs.points(df, 'x', 'y', ds.count('nbre_pdc'))
img = tf.shade(agg, cmap=['lightblue', 'darkblue'], how='log')

img

In [ ]:
from colorcet import cm, fire
from geoviews.tile_sources import EsriImagery
import cartopy.crs as ccrs

In [ ]:
opts = dict(width=600, height=600, xaxis=None, yaxis=None, bgcolor='white')

dataset = gv.Dataset(df, kdims=['Xlongitude', 'Ylatitude'], vdims=['nbre_pdc'])
points = hv.Points(df, ['x', 'y'], ['nbre_pdc'])
#decimate(points.options(**opts))

pdc = datashade(points, cmap=fire, aggregator=ds.sum('nbre_pdc')).options(**opts)

print(points)
(pdc * EsriImagery) + (dynspread(pdc) * EsriImagery)

In [ ]:
France = ((-6.152344, 8.701172), (42.504503, 51.713416))
x_range,y_range = [list(r) for r in ds.utils.lnglat_to_meters(*France)]

cvs = ds.Canvas(plot_width=800, plot_height=800, x_range=x_range, y_range=y_range)
agg = cvs.points(df, 'x', 'y', ds.count('nbre_pdc'))
img = tf.shade(agg, cmap=['lightblue', 'darkblue'], how='eq_hist')

img

In [ ]:
shapefile = '/tmp/out/departements-20140306-100m.shp'
departements = gv.Shape.from_shapefile(shapefile)

In [ ]:
departements * dynspread(pdc)